In [84]:
import pandas as pd
import database_connect as db_connect
import time
from datetime import timezone
import stock_marker_binance



In [49]:
def hours_to_milliseconds(hours):
    return hours * 60 * 60 * 1000

In [59]:
def insert_data_to_db(conn, df):
    with conn.cursor() as cursor:
        for row in df.itertuples():
            query = f"INSERT INTO trading_bot_historydataeth (timestamp, open, high, low, close, volume, \
            close_time, quote_asset_volume, num_trades) \
            VALUES ('{row.timestamp}', {row.open}, {row.high}, {row.low}, {row.close}, {row.volume}, \
            {row.close_time}, {row.quote_asset_volume}, {row.num_trades})"
            cursor.execute(query)
            conn.commit()

### Adding historical data to the database.

In [ ]:
df = pd.read_csv('ETH_1h_history as of 09-01-2024.csv', index_col=0)

In [25]:
df.head()

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades
0,01/01/2020 00:00,129.16,129.19,128.68,128.87,7769.17336,1.577840e+12,1.000930e+06,2504
1,01/01/2020 01:00,128.87,130.65,128.78,130.64,11344.65516,1.577840e+12,1.474278e+06,4885
2,01/01/2020 02:00,130.63,130.98,130.35,130.85,7603.35623,1.577850e+12,9.940256e+05,3046
3,01/01/2020 03:00,130.85,130.89,129.94,130.20,4968.55433,1.577850e+12,6.473610e+05,2818
4,01/01/2020 04:00,130.21,130.74,130.15,130.20,3397.90747,1.577850e+12,4.430067e+05,2264


In [27]:
config = db_connect.load_config()
conn = db_connect.connect(config)

Connected to the PostgreSQL server.


In [28]:
for row in df.itertuples():
    print(tuple(row))
    break

(0, '01/01/2020 00:00', 129.16, 129.19, 128.68, 128.87, 7769.17336, 1577840000000.0, 1000929.742, 2504)


In [29]:
# insert_data_to_db(conn, df) # Commented to avoid inserting data to db

In [32]:
conn.close()

### Update database with fresh data from Binance

In [ ]:
# Every hour the script updates the database with the latest data from the API.
# The script is run using a cron job.

# Identifying the last timestamp in the database and using that as the starting point for the API call.

In [81]:
# Identifying the last timestamp in the database and using that as the starting point for the API call.
config = db_connect.load_config()
conn = db_connect.connect(config)

Connected to the PostgreSQL server.


In [82]:
with conn.cursor() as cursor:
    cursor.execute("SELECT MAX(timestamp) FROM trading_bot_historydataeth")
    result = cursor.fetchone()
    print(result)
conn.close()

(datetime.datetime(2024, 1, 9, 7, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=18000))),)


datetime.datetime

In [91]:
INTERVAL_START = result[0]

# Adding 1 millisecond to avoid duplicates. Converted to UNIX time format.
INTERVAL_START = int(INTERVAL_START.replace(tzinfo=timezone.utc).timestamp() * 1000) +1




In [93]:
url = 'https://api3.binance.com/api/v3/time'

df_time_stop = pd.read_json(url, typ='series')

TIME_STOP = df_time_stop.iloc[0]
TIME_STOP = int ( time.mktime(TIME_STOP.timetuple())*1000)
print(TIME_STOP)

1707232173000


In [94]:
SYMBOL = 'ETHUSDT'
INTERVAL = '1h'
df_total = pd.DataFrame()

In [95]:
while INTERVAL_START < TIME_STOP:
      df=stock_marker_binance.import_stock_data(SYMBOL, INTERVAL, startTime =INTERVAL_START, limit=984)
      df.rename(
          columns={0:'timestamp', 1:'open', 2:'high', 3:'low', 4:'close',5:'volume', 6:'close_time', 7:'quote_asset_volume',
                8:'num_trades', 9:'taker_buy_base_asset_volume',10:'taker_buy_quote_asset_volume', 11:'unused_field'},
         inplace=True)
      df.drop(columns=['taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'unused_field'], inplace=True)
      if len(df_total)>0:
            df_total = pd.concat([df_total, df], ignore_index=True)
      else:
            df_total = df.copy()
      INTERVAL_START = INTERVAL_START + hours_to_milliseconds(984)
      print(len(df_total))

685


In [96]:
df_total.head()

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades
0,2024-01-09 08:00:00,2306.05,2311.30,2290.26,2291.28,11885.1589,1704790799999,2.734850e+07,29276
1,2024-01-09 09:00:00,2291.28,2297.22,2285.40,2291.37,14051.9445,1704794399999,3.220718e+07,34420
2,2024-01-09 10:00:00,2291.37,2301.73,2289.00,2292.40,9995.6604,1704797999999,2.294682e+07,23497
3,2024-01-09 11:00:00,2292.40,2303.09,2289.14,2298.76,9746.6547,1704801599999,2.238487e+07,25248
4,2024-01-09 12:00:00,2298.77,2304.21,2226.78,2264.20,28207.9071,1704805199999,6.415147e+07,67939


In [98]:
conn = db_connect.connect(config)
insert_data_to_db(conn, df_total)
conn.close()

Connected to the PostgreSQL server.


### Recalculate the additional columns